In [1]:
import pandas as pd
import cv2

from pathlib import Path
import re

TRAIN_DIR = Path("/home/lex/data/Spatial_Monitoring_and_Insect_Behavioural_Analysis_Dataset/YOLOv4_Training_and_Test_Dataset/training")
TEST_DIR = Path("/home/lex/data/Spatial_Monitoring_and_Insect_Behavioural_Analysis_Dataset/YOLOv4_Training_and_Test_Dataset/testing")
# Matches "PDDHHMMSS0I.csv" (not sure about that 0, it's sometimes a 1 instead?)
# FILE_REGEX = re.compile(r"(?P<point>\d)(?P<date>\d{2})(?P<hour>\d{2})(?P<minute>\d{2})(?P<second>\d{2})\d(?P<insect_type>\d)")
INSECT_TXT_REGEX = re.compile(r"insect_(\d+).txt")

# Index as used in list of classes according to README for this data
BEE_INDEX = 0

In [4]:
csv_columns = ["insect_type", "centre_x", "centre_y", "width", "height"]
extra_columns = ["image"]
df = pd.DataFrame(columns=[*extra_columns, *csv_columns])

for file in TRAIN_DIR.iterdir():
    match = INSECT_TXT_REGEX.match(file.name)
    if match:
        row = pd.read_csv(file, sep=" ", names=csv_columns)
        
        # Get corresponding image for this insect 
        image_fp = file.with_suffix(".png")
        row["image"] = image_fp.name
        df = pd.concat([df, row])

In [14]:
INSECT_NUM = "02965"
csv_fp = f"/home/lex/data/Spatial_Monitoring_and_Insect_Behavioural_Analysis_Dataset/YOLOv4_Training_and_Test_Dataset/training/insect_{INSECT_NUM}.txt"
image_fp = f"/home/lex/data/Spatial_Monitoring_and_Insect_Behavioural_Analysis_Dataset/YOLOv4_Training_and_Test_Dataset/training/insect_{INSECT_NUM}.png"

df = pd.read_csv(csv_fp, sep=" ", names=["insect_type", "fx", "fy", "width", "height"])
image = cv2.imread(image_fp)
height, width, num_channels = image.shape

for index, row in df.iterrows():
    x1 = int((row["fx"] - 0.5*row["width"]) * width)
    y1 = int((row["fy"] - 0.5*row["height"]) * height)
    x2 = int((row["fx"] + 0.5*row["width"]) * width)
    y2 = int((row["fy"] + 0.5*row["height"]) * height)
    # x2 = int((row["fx"] + row["width"]) * width)
    # y2 = int((row["fy"] + row["height"]) * height)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

WINDOW_NAME = "Insect"
cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)

while True:
    cv2.imshow(WINDOW_NAME, image)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cv2.destroyAllWindows()

In [5]:
df

,image,insect_type,centre_x,centre_y,width,height
0,insect_02097.png,0,0.438305,0.512250,0.016172,0.033593
0,insect_02905.png,0,0.131612,0.412750,0.011568,0.035130
0,insect_01728.png,0,0.394792,0.306481,0.017708,0.025926
1,insect_01728.png,1,0.294792,0.488889,0.030208,0.055556
2,insect_01728.png,1,0.514062,0.241204,0.038542,0.065741
...,...,...,...,...,...,...
3,insect_01146.png,1,0.452471,0.331269,0.040880,0.070370
4,insect_01146.png,1,0.463844,0.498384,0.039583,0.065750
5,insect_01146.png,1,0.521083,0.721444,0.035687,0.055370
6,insect_01146.png,1,0.582333,0.679495,0.039583,0.065750


In [7]:
len(df["image"].unique())

2997